# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [20]:
# imports

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [21]:
# constants
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'
HEADERS = {"Content-Type": "application/json"}
OLLAMA_API = "http://localhost:11434/api/chat"

openai = OpenAI()

API key looks good so far


In [22]:
system_prompt = "You are an assistant that takes a technical question and respond with an explanation."

In [23]:
def create_user_prompt(question):
    user_prompt = "You are going to answer this technical question.\n"
    user_prompt += question
    user_prompt += "\nProvide the solution with an explanation in markdown."
    return user_prompt
    

In [28]:
# set up environment
def answer_technical_question(question):
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": create_user_prompt(question)},
        ],
        stream = True
    )
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        # response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [25]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [29]:
# Get gpt-4o-mini to answer, with streaming
answer_technical_question(question)

Certainly! Let's break down the code snippet `yield from {book.get("author") for book in books if book.get("author")}`.

### Explanation

1. **Context of `yield from`**: 
   - The `yield from` expression in Python is used within a generator function to yield all values from an iterable. This effectively delegates the generator's operations to another generator or iterable, enabling a clean and efficient way to yield multiple values.

2. **The Set Comprehension**:
   - `{book.get("author") for book in books if book.get("author")}` is a **set comprehension**. Set comprehensions are used to create a new set by iterating over an iterable (in this case, `books`), applying an expression, and optionally filtering the items based on a condition.
   - Here, for each `book` in the `books` iterable, `book.get("author")` is called. This attempts to retrieve the value associated with the key `"author"` from each `book`.
   - The condition `if book.get("author")` filters the books to include only those that have a truthy value for the `"author"` key. In Python, this means it will exclude any books where the `"author"` is either `None`, an empty string, or any other falsey value.

3. **Conversion to a Set**:
   - The result of the set comprehension is a set of unique authors. A set automatically handles duplicates, so if multiple books have the same author, that author will only appear once in the final result.

4. **Putting It All Together**:
   - The entire code line effectively gathers all the unique authors from a collection of books and yields them one by one. 
   - Since the expression is wrapped with `yield from`, if this line is inside a generator function, it allows the function to yield each unique author directly to the caller, streamlining the process compared to yielding each author individually.

### Example

Here is a simple example for clarity:

```python
books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "author": "Author A"},
    {"title": "Book 3", "author": "Author B"},
    {"title": "Book 4", "author": None},
    {"title": "Book 5", "author": "Author C"},
]

def get_unique_authors(books):
    yield from {book.get("author") for book in books if book.get("author")}

# Usage
for author in get_unique_authors(books):
    print(author)
```

### Output

In this case, the output would be:
```
Author A
Author B
Author C
```

### Summary

In summary, the code efficiently extracts unique author names from a list of books while ignoring books without an author. It combines the power of set comprehensions and generator functions in Python to produce a streamlined and efficient solution for obtaining and yielding unique authors.

In [26]:
# Get Llama 3.2 to answer

def answer_technical_question_ollama(question):
    messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": create_user_prompt(question)},
        ]
    payload = {
        "model": MODEL_LLAMA,
        "messages": messages,
        "stream": False
    } 
    response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
    display(Markdown(response.json()['message']['content']))

In [27]:
answer_technical_question_ollama(question)

**Decoupling Looping Over Collections**
======================================

The given code utilizes a feature of Python called **generators and `yield`** to break down a complex loop into smaller, more manageable pieces.

### Code Explanation
```python
yield from {book.get("author") for book in books if book.get("author")}
```
This line is written in a combination of Python syntax. Here's what it does:

*   `{...}`: This is a set comprehension, which creates an iterator that yields elements from the `books` collection.
*   `for book in books`: This part iterates over each item (book) in the `books` collection.
*   `if book.get("author")`: Before yielding the author of a book, this condition filters out any book that does not have an "author" key or value.
*   `yield from {...}`: Instead of explicitly iterating and yielding elements with `.next()`, we use `yield from`. This tells Python to yield all values generated by the inner iterable.

### How It Works

Here's a step-by-step breakdown:

1.  Iterate over each book in the `books` collection.
2.  Filter out any books that do not have an "author" key or value.
3.  For each book with an author, yield their name.

**Example Use Case**

Suppose we have two dictionaries representing books:
```python
books = [
    {"title": "Book A", "author": "John Doe"},
    {"title": "Book B", "author": None},
    {"title": "Book C", "author": "Jane Smith"}
]
```
Running the given code will produce an iterator with the following values:

```python
["John Doe", "Jane Smith"]
```

This is because only books with an author have been included in the output.